# Imports

In [ ]:
# general
import os
import math
import functools
import itertools

import pandas               as pd
import numpy                as np
import seaborn              as sns
import matplotlib.pyplot    as plt
import scipy.stats          as stats

# preprocessing
from sklearn.model_selection    import train_test_split
from sklearn.model_selection    import StratifiedKFold
from sklearn.model_selection    import learning_curve
from sklearn.model_selection    import RandomizedSearchCV
from sklearn.preprocessing      import OneHotEncoder
from sklearn.preprocessing      import StandardScaler
from sklearn.preprocessing      import MinMaxScaler
from sklearn.preprocessing      import RobustScaler
from sklearn.decomposition      import PCA
from sklearn.feature_selection  import RFECV
from sklearn.feature_selection  import SelectFromModel
from sklearn.inspection         import permutation_importance
from sklearn.pipeline           import Pipeline
from sklearn.compose            import ColumnTransformer

# classifier
from sklearn.linear_model       import LogisticRegression
from sklearn.linear_model       import SGDClassifier
from sklearn.svm                import SVC
from sklearn.neural_network     import MLPClassifier
from sklearn.tree               import DecisionTreeClassifier
from sklearn.ensemble           import RandomForestClassifier

# metrics
from sklearn.metrics            import f1_score
from sklearn.model_selection    import cross_val_score
from sklearn.metrics            import RocCurveDisplay
from sklearn.metrics            import PrecisionRecallDisplay

---
# Load Data

In [ ]:
# create dictionary with filename:data as key:value pairs
data = {dataset_name.replace('.xlsx', ''): pd.read_excel(f'Data/{dataset_name}') for dataset_name in os.listdir('Data') if 'xlsx' in dataset_name}

# merge train datasets on PatientID
data_frames = [data['train_demo'], data['train_habits'], data['train_health']]
df = functools.reduce(lambda  left,right: pd.merge(left,right,on=['PatientID'], how='outer'), data_frames).set_index('PatientID')

# merge test datasets on PatientID
data_frames = [data['test_demo'], data['test_habits'], data['test_health']]
df_test = functools.reduce(lambda  left,right: pd.merge(left,right,on=['PatientID'], how='outer'), data_frames).set_index('PatientID')

---
# Exploratory Data Analysis

In [ ]:
# first look on the data
df.sample(5)

In [ ]:
# list datatypes
df.dtypes

In [ ]:
# check for missing values and empty strings
pd.concat([df.isnull().sum(),df.eq('').sum()],keys=['Nulls','Empty Strings'],axis=1)

In [ ]:
# check for duplicated rows
df.duplicated().sum()

In [ ]:
# descriptive statistics
df.describe(include="all").T

In [ ]:
sns.set()

# plot pairwise relationships and densities
sns.pairplot(df, hue = 'Disease', markers = ['o', 's'])
plt.show()

In [ ]:
# select numeric features (exclude target array)
df_numeric_features = list(df.select_dtypes(include = np.number).columns)
df_numeric_features.remove('Disease')

# boxplot of numeric features
fig, ax = plt.subplots(math.ceil(len(df_numeric_features)/4),4, figsize = (15,10))
for ax, feat in zip(ax.flatten(), df_numeric_features):
    ax.boxplot(df[feat], notch = True, patch_artist = True)
    ax.set_title(feat)

plt.show()

In [ ]:
# select categorical features (exclude name column)
df_categorical_features = list(df.select_dtypes(exclude = np.number).columns)
df_categorical_features.remove('Name')

# stacked barplot of categorical features (with regard to target value)
fig, ax = plt.subplots(len(df_categorical_features), figsize = (10,50))
for ax, feat in zip(ax.flatten(), df_categorical_features):
    pivot_tbl = df[[feat, 'Disease']].pivot_table(index = feat, columns = ['Disease'],  aggfunc=len)
    graph = pivot_tbl.plot(kind='barh', stacked=True, title=feat, ax=ax)
    ax.set_ylabel('')
    for c in ax.containers:
        ax.bar_label(c, label_type='center')

plt.show()

In [ ]:
# plot gender distribution
gender_imbalance = df.copy()
gender_imbalance['Gender'] = ['Men' if i.split(' ')[0] == 'Mr.' else 'Women' for i in gender_imbalance.Name]
pivot_tbl = gender_imbalance[['Gender', 'Disease']].pivot_table(index = 'Gender', columns = ['Disease'],  aggfunc=len)

ax = pivot_tbl.plot(kind='bar', stacked=True, title='Gender Imbalance')

for container in ax.containers:
    ax.bar_label(container, label_type='center')

plt.show()

---
# Baseline Model

- LogisticRegression with only numeric values

In [ ]:
# split to data & target
x, y = df.drop(columns = ['Disease']), df['Disease']

In [ ]:
# select numeric features
initial_xnumeric = x.select_dtypes(include = np.number)
initial_xnumeric_cols = initial_xnumeric.columns

# stratified train-test split
xtrain, xval, ytrain, yval = train_test_split(initial_xnumeric, y, random_state = 0 ,test_size = 0.2, shuffle = True , stratify = y)

In [ ]:
# very basic baseline model
model = LogisticRegression(max_iter = 500, random_state = 1)
model.fit(xtrain,ytrain)

ypred = model.predict(xval)

baseline_f1 = f1_score(yval, ypred)

print(f'F1 score: {baseline_f1}')

---
# Preprocessing

In [ ]:
def preprocessing(dataframe):

    # fix Birth_Year errors. here we assume that the outliers are typos
    # the numbers 8 & 9 are pretty close to each other on a keyboard. therefore we add 100 to every year that's smaller than 1900
    # (e.g. 1869 --> 1969)
    dataframe['Birth_Year'] = [i + 100 if i < 1900 else i for i in dataframe['Birth_Year']]

    # add Age column
    dataframe['Age'] = [2022 - i for i in dataframe['Birth_Year']]

    # add Gender column (1: Male, 0: Female)
    # afterwards drop column Name
    dataframe['Gender']  = [1 if i.split(' ')[0] == 'Mr.' else 0 for i in dataframe['Name']]
    dataframe.drop(columns = ['Name'], inplace = True)

    # add BMI columns
    dataframe['BMI'] = [i/(j/100)**2 for i, j in zip(dataframe['Weight'], dataframe['Height'])]
    dataframe['BMI_Cateogry'] = [0 if i < 18.5 else 1 if i < 25 else 2 if i < 30 else 3 if i < 35 else 4 for i in dataframe['BMI']]

    # set all region values to lowercase
    dataframe['Region'] = [i.lower() for i in dataframe['Region']]

    # handle High_Cholesterol, Blood_Pressure, Physical_Health
    # flooring and capping outliers
    for col in ['High_Cholesterol', 'Blood_Pressure', 'Physical_Health']:
        q25, q75, iqr = dataframe[col].quantile(.25), dataframe[col].quantile(.75), dataframe[col].quantile(.75) - dataframe[col].quantile(.25)
        upper_lim = q75 + 1.5 * iqr
        lower_lim = q25 - 1.5 * iqr
        dataframe[col] = [upper_lim if i > upper_lim else lower_lim if i < lower_lim else i for i in dataframe[col]]

    # encode Smoking_Habit & Exercise to binary (1: Yes, 0: No)
    dataframe['Smoking_Habit'] = [1 if i == 'Yes' else 0 for i in dataframe['Smoking_Habit']]
    dataframe['Exercise'] = [1 if i == 'Yes' else 0 for i in dataframe['Exercise']]    
    
    # manual encoding of specific feature to don't mess up the ranking
    # all of these features have a specific ranking structure
    # another advantage is the avoidance of dimensionality increase through One-hot-encoding all categorical features

    # impute missing values in column "Education" with mode
    # encode Education
    dataframe['Education'].fillna(dataframe['Education'].mode()[0], inplace=True)
    edu_map = {
            'I never attended school / Other'               : 0,
            'Elementary School (1st to 9th grade)'          : 1,
            'High School Incomplete (10th to 11th grade)'   : 2,
            'High School Graduate'                          : 3,
            'University Incomplete (1 to 2 years)'          : 4,
            'University Complete (3 or more years)'         : 5
            }
    dataframe['Education'] = [edu_map[i] if i in edu_map.keys() else np.nan for i in dataframe['Education']]

    drink_map = {
            'I do not consume any type of alcohol'          : 0,
            'I consider myself a social drinker'            : 1,
            'I usually consume alcohol every day'           : 2
            }
    dataframe['Drinking_Habit'] = [drink_map[i] if i in drink_map.keys() else np.nan for i in dataframe['Drinking_Habit']]

    fruit_map = {
            'Less than 1. I do not consume fruits every day.'   : 0,
            '1 to 2 pieces of fruit in average'                 : 1,
            '3 to 4 pieces of fruit in average'                 : 2,
            '5 to 6 pieces of fruit in average'                 : 3,
            'More than six pieces of fruit'                     : 4
            }
    dataframe['Fruit_Habit'] = [fruit_map[i] if i in fruit_map.keys() else np.nan for i in dataframe['Fruit_Habit']]

    water_map = {
            'Less than half a liter'                            : 0,
            'More than half a liter but less than one liter'    : 1,
            'Between one liter and two liters'                  : 2
            }
    dataframe['Water_Habit'] = [water_map[i] if i in water_map.keys() else np.nan for i in dataframe['Water_Habit']]

    checkup_map = {
            'Not sure'                                          : 0,
            'More than 3 years'                                 : 1,
            'Less than 3 years but more than 1 year'            : 2,
            'Less than three months'                            : 3
            }
    dataframe['Checkup'] = [checkup_map[i] if i in checkup_map.keys() else np.nan for i in dataframe['Checkup']]

    diabetes_map = {
            'I do have diabetes'                                                            : 0,
            'I have/had pregnancy diabetes or borderline diabetes'                          : 1,
            "I don't have diabetes, but I have direct family members who have diabetes."    : 2,
            'Neither I nor my immediate family have diabetes.'                              : 3
            }
    dataframe['Diabetes'] = [diabetes_map[i] if i in diabetes_map.keys() else np.nan for i in dataframe['Diabetes']]

    return dataframe

In [ ]:
# apply preprocessing steps to x
x_preprocessed = preprocessing(x.copy())

---
# Check Different Scaler Options

In [ ]:
# scaler options
scalers = [StandardScaler(), MinMaxScaler(), MinMaxScaler(feature_range = (-1,1)), RobustScaler()]

In [ ]:
def test_scaler_encoder(X,Y,scaler_options,classifier):

        ranking = {'scaler':[],'score':[]}

        # create crossvalidation splits
        skf = StratifiedKFold(shuffle=True, random_state = 0)

        for scl in scaler_options:

                # scores list for every k-fold f1-score
                scores = []

                # calculate f1-score for each train-test split with the given scaler
                for train_index, val_index in skf.split(X, Y):
                        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
                        y_train, y_val = Y.iloc[train_index], Y.iloc[val_index]

                        numeric_features, categorical_features = X_train.select_dtypes(include = np.number).columns, X_train.select_dtypes(exclude = np.number).columns

                        numeric_transformer = Pipeline(steps=[
                                ('scaler', scl)
                                ])

                        categorical_transformer = Pipeline(steps=[
                                ('encoder', OneHotEncoder(handle_unknown='ignore'))
                                ])

                        preprocessor = ColumnTransformer(transformers=[
                                ('cat', categorical_transformer, categorical_features),
                                ('num', numeric_transformer, numeric_features)
                                ])

                        model = Pipeline(steps=[
                                ("preprocessor", preprocessor),
                                ("classifier", classifier)
                                ])
                        
                        model.fit(X_train, y_train)
                        # add every score to the previously created scores list
                        scores.append(round(f1_score(y_val, model.predict(X_val)),4))

                # add scaler and cross validated mean of scores list to dictionary
                ranking['scaler'].append(scl)
                ranking['score'].append(np.mean(scores))

        # returns dataframe with the cross validated scores for each scaler
        return pd.DataFrame(ranking).sort_values('score', ascending=False)

In [ ]:
test_scaler_encoder(x_preprocessed, y, scalers, LogisticRegression(max_iter=500, random_state=1))

In [ ]:
test_scaler_encoder(x_preprocessed, y, scalers, SGDClassifier(random_state=1))

In [ ]:
test_scaler_encoder(x_preprocessed, y, scalers, SVC(random_state=1))

In [ ]:
test_scaler_encoder(x_preprocessed, y, scalers, MLPClassifier(max_iter=1500, random_state=1))

In [ ]:
test_scaler_encoder(x_preprocessed, y, scalers, DecisionTreeClassifier(random_state=1))

In [ ]:
test_scaler_encoder(x_preprocessed, y, scalers, RandomForestClassifier(random_state=1))

---
# Train/Test Split

In [ ]:
# again a stratified train-test split with the same random_state as before
xtrain, xval, ytrain, yval = train_test_split(x_preprocessed, y, random_state = 0 ,test_size = 0.2, shuffle = True , stratify = y)

---
# Prepare Train/Test Split (One-Hot-Encode, Scale) 

In [ ]:
def encode_scale(dataframe, reset_fit=False):

    '''
    One-hot-encode and scales dataframe
    If reset_fit set to TRUE encoder & scaler get refitted
    '''

    # fit_transform if no scaler and one-hot-encoder is already fitted
    if not('ohe' in globals() and 'scl' in globals()) or reset_fit == True:
        xnumeric, xcategorical = dataframe.select_dtypes(include = np.number), dataframe.select_dtypes(exclude = np.number)
        # One-hot-encoding
        # set encoder to global variable to re-use it afterwards
        global ohe
        ohe = OneHotEncoder(handle_unknown='ignore')
        categorical_encoded = pd.DataFrame(ohe.fit_transform(xcategorical).toarray(), columns = ohe.get_feature_names_out(), index = xcategorical.index)
        dataframe_encoded = pd.concat([xnumeric, categorical_encoded], axis = 1)
        # Robust-scaling
        # set scaler to global variable to re-use it afterwards
        global scl
        scl = RobustScaler()
        dataframe_scaled = pd.DataFrame(scl.fit_transform(dataframe_encoded), columns = dataframe_encoded.columns, index = dataframe_encoded.index)
        print('fit & transform successful...')
        return dataframe_scaled

    # transform only if scaler and one-hot-encoder is already fitted
    else:
        xnumeric, xcategorical = dataframe.select_dtypes(include = np.number), dataframe.select_dtypes(exclude = np.number)
        # One-hot-encoding
        categorical_encoded = pd.DataFrame(ohe.transform(xcategorical).toarray(), columns = ohe.get_feature_names_out(), index = xcategorical.index)
        dataframe_encoded = pd.concat([xnumeric, categorical_encoded], axis = 1)
        #Robust-scaling
        dataframe_scaled = pd.DataFrame(scl.transform(dataframe_encoded), columns = dataframe_encoded.columns, index = dataframe_encoded.index)

        print('transform successful...')
        return dataframe_scaled

In [ ]:
# apply encoding and scaling function on xtrain (fit & transform)
xtrain_prepro = encode_scale(xtrain, reset_fit=True)

In [ ]:
# apply encoding and scaling function on xtrain (only transform)
xval_prepro = encode_scale(xval)

---
# PCA

In [ ]:
def principal_component_analysis(dataframe, var_treshold=0.8, reset_fit=False):
    '''
    Creates and add n principal components (dependent on variance treshold)
    '''

    # fit_transform if pca isn't already fitted
    if not 'pca' in globals() or reset_fit == True:
        # set 'pca' to global variable to re-use it afterwards
        global pca
        pca = PCA(n_components=var_treshold)
        pca_feat = pca.fit_transform(dataframe)
        pca_feat_names = [f'PC{i}' for i in range(pca.n_components_)]
        pca_df = pd.DataFrame(pca_feat, index=dataframe.index, columns=pca_feat_names)
        dataframe = pd.concat([dataframe, pca_df], axis=1)
        print('fit & transform successful...')
        return dataframe
    
    # transform only if pca is already fitted
    else:
        pca_feat = pca.transform(dataframe)
        pca_feat_names = [f'PC{i}' for i in range(pca.n_components_)]
        pca_df = pd.DataFrame(pca_feat, index=dataframe.index, columns=pca_feat_names)
        dataframe = pd.concat([dataframe, pca_df], axis=1)
        print('transform successful...')
        return dataframe

In [ ]:
# add pca features to xtrain (fit & transform)
xtrain_pca = principal_component_analysis(xtrain_prepro)

In [ ]:
# add pca features to xval (only transform)
xval_pca = principal_component_analysis(xval_prepro)

---
# Feature Selection - Filter Methods

## Correlation Matrix (Spearman)

In [ ]:
def cor_heatmap(cor):
    plt.figure(figsize=(10,7))
    sns.heatmap(data = cor, annot = True, cmap = plt.cm.Reds, fmt='.2')
    plt.show()

cor_spearman = cor_heatmap(xtrain_prepro[initial_xnumeric_cols].corr("spearman"))

## Chi-squared

In [ ]:
def TestIndependence(X,Y,var,alpha=0.05):
    CRED = '\033[91m'
    CGREEN = '\033[92m'
    CEND = '\033[0m'   
    dfObserved = pd.crosstab(Y,X) 
    chi2, p, dof, expected = stats.chi2_contingency(dfObserved.values)
    dfExpected = pd.DataFrame(expected, columns=dfObserved.columns, index = dfObserved.index)
    if p<alpha:
        result= str(var) + CGREEN + ' IMPORTANT' + CEND
    else:
        result= str(var) + CRED + ' NOT IMPORTANT' + CEND
    print(result)

In [ ]:
initial_xcategorical_col = [i for i in x.columns if i not in initial_xnumeric_cols]
initial_xcategorical_col.remove('Name')
initial_xcategorical_col.remove('Region')

for var in initial_xcategorical_col:
    TestIndependence(xtrain_prepro[var],ytrain, var)

# Feature Selection - Wrapper Methods

## Recursive Feature Elimination (RFE)

In [ ]:
model = RandomForestClassifier(random_state = 1)
rfe = RFECV(model, cv = 5, scoring = 'f1', n_jobs = -1)
rfe.fit(xtrain_prepro, ytrain)
opt_features = list(rfe.get_feature_names_out(input_features = list(xtrain_prepro.columns)))
print(f'Best features: {[i for i in opt_features]}')

In [ ]:
# transform data to RFE results
xtrain_rfe = rfe.transform(xtrain_prepro)
xval_rfe = rfe.transform(xval_prepro)

# Feature Selection - Embedded Method

## Select From Model

In [ ]:
# embedded feature selection with SelectFromModel and RandomForestClassifier
sfm = SelectFromModel(RandomForestClassifier(random_state = 1))
sfm.fit(xtrain_prepro, ytrain)

In [ ]:
# check columns
selected_feat = xtrain_prepro.columns[(sfm.get_support())]
selected_feat

In [ ]:
# transform data to SelectFromModel results
xtrain_sfm = pd.DataFrame(sfm.transform(xtrain_prepro), columns=xtrain_prepro.columns[(sfm.get_support())], index=xtrain_prepro.index)
xval_sfm = pd.DataFrame(sfm.transform(xval_prepro), columns=xval_prepro.columns[(sfm.get_support())], index=xval_prepro.index)

---
# Model Selection

In [ ]:
classifiers = {
                'Logistic Regression': LogisticRegression(),
                'Support Vector Machine': SVC(),
                'Stochastic Gradient Descent': SGDClassifier(),
                'Neural Network': MLPClassifier(max_iter = 1500),
                'Decision Tree': DecisionTreeClassifier(),
                'Random Forest': RandomForestClassifier()
                }

In [ ]:
def roc_auc(classifiers, xtrain, ytrain, xval, yval):
    '''
    Plots ROC/AUC
    classifiers input --> {'Logistic Regression': LogisticRegression(),...}
    '''

    fig, ax = plt.subplots(1, figsize=(15, 10))
    for name, clf in classifiers.items():
        clf.fit(xtrain, ytrain)
        RocCurveDisplay.from_estimator(clf, xval, yval, ax=ax, name=name)
    ax.set_title('ROC/AUC Curve')
    ax.plot([0,1], [0,1], linestyle='--')
    plt.show()

In [ ]:
def prec_rec(classifiers, xtrain, ytrain, xval, yval):
    '''
    Plots Precision/Recall Curve
    classifiers input --> {'Logistic Regression': LogisticRegression(),...}
    '''

    fig, ax = plt.subplots(1, figsize=(15, 10))
    for name, clf in classifiers.items():
        clf.fit(xtrain, ytrain)
        PrecisionRecallDisplay.from_estimator(clf, xval, yval, ax=ax, name=name)
    ax.set_title('Precision/Recall Curve')
    ax.plot([0,1], [1,0], linestyle='--')
    plt.show()

In [ ]:
def learning_curves(estimator, X, Y):
    '''
    Plots cross validated learning curve
    '''

    train_sizes, train_scores, validation_scores = learning_curve(estimator, X, Y, cv = 5, scoring = 'f1', train_sizes = np.arange(.05,1,.05))
    train_mean, test_mean, train_std, test_std = np.mean(train_scores, axis=1), np.mean(validation_scores, axis=1), np.std(train_scores, axis=1), np.std(validation_scores, axis=1)

    plt.subplots(1, figsize=(10,10))
    plt.plot(train_sizes, train_mean, color='salmon',  label='Training score', marker = 'o')
    plt.plot(train_sizes, test_mean, color='olive', label='Cross-validation score', marker = 's')
    plt.title('Learning Curve')
    plt.xlabel('Training Set Size')
    plt.ylabel('F1 Score')
    plt.legend(loc='best')
    plt.show()

## ROC/AUC Curve

In [ ]:
roc_auc(classifiers, xtrain_sfm, ytrain, xval_sfm, yval)

## Precision/Recall Curve

In [ ]:
prec_rec(classifiers, xtrain_sfm, ytrain, xval_sfm, yval)

---
# Model Optimization


## Stochastic Gradient Descent (SGDClassifier) - Optimizing

In [ ]:
# feature selection with SelectFromModel (SGDClassifier)
sfm = SelectFromModel(SGDClassifier(random_state = 1))
sfm.fit(xtrain_prepro, ytrain)

xtrain_sfm = pd.DataFrame(sfm.transform(xtrain_prepro), columns=xtrain_prepro.columns[(sfm.get_support())], index=xtrain_prepro.index)
xval_sfm = pd.DataFrame(sfm.transform(xval_prepro), columns=xval_prepro.columns[(sfm.get_support())], index=xval_prepro.index)

In [ ]:
# get first score of model
model = SGDClassifier(random_state = 1)
model.fit(xtrain_sfm, ytrain)

ypred = model.predict(xval_sfm)

print(f'F1 Score: {f1_score(yval, ypred)}')

In [ ]:
# checking the learning curve for over-fitting
learning_curves(SGDClassifier(random_state=1), xtrain_sfm, ytrain)

In [ ]:
# performing hyperparameter tuning
param_grid = {
    'loss': ['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron'],
    'penalty': ['none', 'l2', 'l1', 'elasticnet'],
    'alpha': [0.0001, 0.001, 0.01, 0.1],
    'l1_ratio': [0, 0.15, 0.5, 0.85, 1]
}

model = SGDClassifier(random_state = 1)
rand_search = RandomizedSearchCV(estimator = model, param_distributions = param_grid, scoring = 'f1', n_iter = 100, cv = 3, random_state=3, n_jobs = -1)
rand_search.fit(xtrain_sfm, ytrain)

print(str(rand_search.best_params_).replace('{','').replace('}','').replace("'","").replace(':','='))

In [ ]:
# check learning curve again
learning_curves(SGDClassifier(**rand_search.best_params_, random_state=1), xtrain_sfm, ytrain)

In [ ]:
# get final score with tuned hyperparameters
model = SGDClassifier(**rand_search.best_params_, random_state = 1)
model.fit(xtrain_sfm, ytrain)

ypred = model.predict(xval_sfm)

print(f'F1 Score: {f1_score(yval, ypred)}')

---
## Neural Network - Optimizing

In [ ]:
# feature selection with permutation_importance (MLPClassifier)
model = MLPClassifier(max_iter=1500, random_state=1)
model.fit(xtrain_prepro, ytrain)

permutation_score = permutation_importance(model, xtrain_prepro, ytrain, n_repeats=10) 

# returns dataframe sorted by permutation score
importance_df = pd.DataFrame(np.vstack((xtrain_prepro.columns, permutation_score.importances_mean)).T, columns=['Feature', 'Score']) 
importance_df = importance_df.sort_values('Score', ascending = False).reset_index(drop=True)
importance_df

In [ ]:
# check which n-best features result in the best cross validated score
results = {'Feature Combination':[],'AVG Score':[]}
for i in range(0,len(importance_df)+1):
    features = list(importance_df.Feature.loc[0:i])
    avg_score = np.mean(cross_val_score(MLPClassifier(max_iter=2000, random_state=1), xtrain_prepro[features], ytrain, cv=3))
    results['Feature Combination'].append(features)
    results['AVG Score'].append(avg_score)

feature_comb_scores = pd.DataFrame(results)
best_comb = feature_comb_scores.sort_values('AVG Score', ascending=False).iloc[0]['Feature Combination']

feature_comb_scores

In [ ]:
# drop other features
xtrain_permut = xtrain_prepro[best_comb]
xval_permut = xval_prepro[best_comb]

In [ ]:
# get first score of model
model = MLPClassifier(max_iter = 1500, random_state = 1)
model.fit(xtrain_permut, ytrain)

ypred = model.predict(xval_permut)

print(f'F1 Score: {f1_score(yval, ypred)}')

In [ ]:
# checking the learning curve for over-fitting
learning_curves(MLPClassifier(max_iter = 1500, random_state = 1), xtrain_permut, ytrain)

In [ ]:
# performing hyperparameter tuning
param_grid = {
    'activation': ['identity', 'logistic', 'tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.001, 0.005, 0.01, 0.05, 0.1],
    'learning_rate': ['constant', 'invscaling', 'adaptive']
}

model = MLPClassifier(max_iter=1500, random_state=1)
rand_search = RandomizedSearchCV(estimator = model, param_distributions = param_grid, scoring = 'f1', n_iter = 100, cv = 3, random_state=3, n_jobs = -1)
rand_search.fit(xtrain_permut, ytrain)

In [ ]:
# print best paramater combinations
print(str(rand_search.best_params_).replace('{','').replace('}','').replace("'","").replace(':','='))

In [ ]:
# check learning curve again
learning_curves(MLPClassifier(max_iter=1500 , **rand_search.best_params_, random_state=1), xtrain_permut, ytrain)

In [ ]:
# get final score with tuned hyperparameters
model = MLPClassifier(max_iter=1500, **rand_search.best_params_, random_state=1)
model.fit(xtrain_permut, ytrain)

ypred = model.predict(xval_permut)

print(f'F1 Score: {f1_score(yval, ypred)}')

---
## Decision Tree - Optimizing

In [ ]:
# feature selection with SelectFromModel (DecisionTreeClassifier)
sfm = SelectFromModel(DecisionTreeClassifier(random_state = 1))
sfm.fit(xtrain_prepro, ytrain)

xtrain_sfm = pd.DataFrame(sfm.transform(xtrain_prepro), columns=xtrain_prepro.columns[(sfm.get_support())], index=xtrain_prepro.index)
xval_sfm = pd.DataFrame(sfm.transform(xval_prepro), columns=xval_prepro.columns[(sfm.get_support())], index=xval_prepro.index)

In [ ]:
# get first score of model
model = DecisionTreeClassifier(random_state = 1)
model.fit(xtrain_sfm, ytrain)

ypred = model.predict(xval_sfm)

print(f'F1 Score: {f1_score(yval, ypred)}')

In [ ]:
# checking the learning curve for over-fitting
learning_curves(DecisionTreeClassifier(random_state=1), xtrain_sfm, ytrain)

In [ ]:
# performing hyperparameter tuning
param_grid = {
    'max_depth': [5, 10, 15, 20, 25],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 3],
    'max_features': ['sqrt', 'log2', None]
}


model = DecisionTreeClassifier(random_state = 1)
rand_search = RandomizedSearchCV(estimator = model, param_distributions = param_grid, scoring = 'f1', n_iter = 100, cv = 3, random_state=3, n_jobs = -1)
rand_search.fit(xtrain_sfm, ytrain)

In [ ]:
# print best paramater combinations
print(str(rand_search.best_params_).replace('{','').replace('}','').replace("'","").replace(':','='))

In [ ]:
# check learning curve again
learning_curves(DecisionTreeClassifier(**rand_search.best_params_, random_state=1), xtrain_sfm, ytrain)

In [ ]:
# get final score with tuned hyperparameters
model = DecisionTreeClassifier(**rand_search.best_params_, random_state = 1)
model.fit(xtrain_sfm, ytrain)

ypred = model.predict(xval_sfm)

print(f'F1 Score: {f1_score(yval, ypred)}')

---
## Random Forest - Optimizing

In [ ]:
# feature selection with SelectFromModel (RandomForestClassifier)
sfm = SelectFromModel(RandomForestClassifier(random_state = 1))
sfm.fit(xtrain_prepro, ytrain)

xtrain_sfm = pd.DataFrame(sfm.transform(xtrain_prepro), columns=xtrain_prepro.columns[(sfm.get_support())], index=xtrain_prepro.index)
xval_sfm = pd.DataFrame(sfm.transform(xval_prepro), columns=xval_prepro.columns[(sfm.get_support())], index=xval_prepro.index)

In [ ]:
# get first score of model
model = RandomForestClassifier(random_state = 1)
model.fit(xtrain_sfm, ytrain)

ypred = model.predict(xval_sfm)

print(f'F1 Score: {f1_score(yval, ypred)}')

In [ ]:
# checking the learning curve for over-fitting
learning_curves(RandomForestClassifier(random_state = 1), xtrain_sfm, ytrain)

In [ ]:
# performing hyperparameter tuning
param_grid = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [5, 10, 15, 20, 25],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 3],
    'max_features': ['sqrt', 'log2', None]
}

model = RandomForestClassifier(random_state = 1)
rand_search = RandomizedSearchCV(estimator = model, param_distributions = param_grid, scoring = 'f1', n_iter = 100, cv = 3, random_state=3, n_jobs = -1)
rand_search.fit(xtrain_sfm, ytrain)

In [ ]:
# print best paramater combinations
print(str(rand_search.best_params_).replace('{','').replace('}','').replace("'","").replace(':','='))

In [ ]:
# check learning curve again
learning_curves(RandomForestClassifier(**rand_search.best_params_, random_state=1), xtrain_sfm, ytrain)

In [ ]:
# get final score with tuned hyperparameters
model = RandomForestClassifier(**rand_search.best_params_, random_state = 1)
model.fit(xtrain_sfm, ytrain)

ypred = model.predict(xval_sfm)

print(f'F1 score: {f1_score(yval, ypred)}')

---
# Submission 

**1** - Train on full dataset

In [ ]:
# apply encoding & scaling
# reset_fit set to True for new fit_transform (including all train observations)
xtrain_prepro_full = encode_scale(preprocessing(x), reset_fit = True)

In [ ]:
# apply feature selection on full train dataset
xtrain_sfm_full = sfm.transform(xtrain_prepro_full)

In [ ]:
# fit model with best hyperparameters
model = RandomForestClassifier(**rand_search.best_params_, random_state = 1)
model.fit(xtrain_sfm_full, y)

**2** - Prepare test dataset and predict

In [ ]:
# apply preprocessing function and encode_scale function to test dataset
xtest_prepro = encode_scale(preprocessing(df_test))

In [ ]:
# transform data to SelectFromModel
xtest_sfm = sfm.transform(xtest_prepro)

In [ ]:
# final predict
ypred = model.predict(xtest_sfm)

In [ ]:
df_submission = pd.concat([pd.Series(df_test.index),pd.Series(ypred)], axis = 1)
df_submission.rename(columns = {0:'Disease'}, inplace = True)
df_submission

In [ ]:
df_submission.to_csv('Group01_Final_RandomForest.csv', index = False)